- Genie Top200 chart

In [2]:
import requests
import pandas as pd

### 1. 인터넷에서 데이터 가져오기

In [3]:
# Genie top200
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
#html

In [4]:
# Chrome User-Agent
header = {'User-Agent':
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
}
req = requests.get(url, headers=header)
html = req.text
# html

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

### 2. 찾으려고 하는 데이터의 노드 찾기

In [6]:
trs = soup.find_all('tr')
len(trs)

51

In [7]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.select('tr')
len(trs)

50

In [8]:
# 개발자 도구에서 찾아주는 Selector
tbody = soup.select_one('#body-content > div.newest-list > div > table > tbody')
# f12에서 copy selector로 찾음
trs = tbody.select('tr')
len(trs)

50

- copy 를 하면 값은 아래와 같으나 BeautifulSoup은 지원하지 않음
<br>
 //*[@id="body-content"]/div[6]/div/table/tbody</br>

### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추출
- rank, title, artist, album

In [9]:
tr  = trs[0]

In [10]:
# rank
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [11]:
rank = tr.select_one('.number').get_text().split('\n')[0]
rank

'1'

In [12]:
# title
tr.select_one('.title.ellipsis').get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                리무진 (Feat. MINO) (Prod. by GRAY)'

In [13]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'리무진 (Feat. MINO) (Prod. by GRAY)'

In [14]:
# artist
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

"BE'O (비오)"

In [15]:
# album
album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'쇼미더머니 10 Episode 3'

### 4. 한 페이지에 있는 모든 데이터를 반복문으로 가져와 데이터프레임 만들기

1. 개별 리스트
rank_list, title_list, 
'순위' = rank_list,
2. 딕셔너리의 리스트
{'순위':rank, }

In [16]:
rank_list, title_list, artist_list, album_list = [],[],[],[]
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

In [17]:
df = pd.DataFrame({
    '순위': rank_list, '제목': title_list, '가수': artist_list, '앨범': album_list
})
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


In [18]:
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([rank, title, artist, album])

df = pd.DataFrame(lines, columns=['순위', '제목', '가수', '앨범'])
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


In [19]:
lines = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    d = {'순위':rank, '제목':title, '가수':artist, '앨범':album}
    lines.append(d)

df = pd.DataFrame(lines)
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


### 5. 모든 페이지의 데이터 가져오기

In [20]:
lines = []
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20211206&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    tbody = soup.select_one('table.list-wrap').find('tbody')
    trs = tbody.select('tr')

    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0]
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
        lines.append([rank, title, artist, album])

In [21]:
df = pd.DataFrame(lines, columns=['순위', '제목', '가수', '앨범'])
df.tail()

,순위,제목,가수,앨범
195,196,우산이 없어요,김호중,우리家
196,197,Coin,아이유 (IU),IU 5th Album 'LILAC'
197,198,내 마음이 움찔했던 순간 (취향저격 그녀 X 규현),규현 (KYUHYUN),내 마음이 움찔했던 순간 (취향저격 그녀 X 규현)
198,199,좋은 사람 있으면 소개시켜줘,조이 (JOY),슬기로운 의사생활 OST Part 2 (tvN 목요드라마)
199,200,I'm a 빛,화사 (Hwa Sa),Guilty Pleasure


In [22]:
df.to_csv('지니차트_Top200(211206).csv', index=False)